# Blind kildeseparasjon med uavhengig komponentanalyse

Prosjekt 1 i TMA4320 våren 2019 - av Thorvald Ballestad, Jonas Bueie og Herman Sletmoen (gruppe 3)

*Blind kildeseparasjon* (engelsk: *blind source separation*) er separasjon av en mengde kildesignaler fra observerte blandinger av disse, uten informasjon om kildesignalene og blandingsprosessen.

*Cocktailparty-problemet* er et konkret eksempel på blind kildeseparasjon.
Betrakt et område med $n$ lydkilder som ved tiden $t$ spiller av lydsignaler med amplituder $s_1(t), \ldots, s_n(t)$, samtidig som $n$ mikrofoner tar opp lydsignaler med amplituder $x_1(t), \ldots, x_n(t)$.
Mikrofonene registrerer totale signaler $x_i(t) = a_{i1} s_1(t) + \ldots + a_{in} s_n(t)$ som lineære kombinasjoner av kildesignalene, der $a_{ij}$ er en antatt tidsuavhengig faktor som bestemmer bidraget fra kildesignalet $s_j(t)$ til totalsignalet $x_i(t)$.
Faktorene $a_{ij}$ avhenger gjerne av geometri, og vil i et perfekt fysisk scenario være omvendt proporsjonale med kvadratet av avstanden $r_{ij}$ mellom lydkilden $s_j(t)$ og mikrofonen $x_i(t)$, som kjent fra klassisk fysikk.

Med kolonnevektorene $\boldsymbol{x} = [x_1(t), \ldots, x_n(t)]^T$ og $\boldsymbol{s} = [s_1(t), \ldots, s_n(t)]^T$ og *miksematrisen* $A = [a_{ij}]$ av størrelse $n \times n$, kan vi mer konsist skrive

$$\boldsymbol{x} = A \boldsymbol{s}$$

I en typisk situasjon vil mikrofonene sample totalsignalene $x_i(t_j)$ ved $N$ diskrete tidspunkter $t_1, \ldots, t_N$, og vi ønsker å bestemme kildesignalene $s_i(t_j)$ ved de samme tidspunktene som bygger opp totalsignalet.
Vektorligningen over vil da gjelde ved ethvert tidspunkt $t_j$.
Med totalsignalsmatrisen $X = [x_{ij}] = [x_i(t_j)]$ og kildsignalsmatrisen $S = [s_{ij}] = [s_i(t_j)]$, begge av størrelse $n \times N$, kan vi dermed oppsummere med matriseligningen

$$X = A S$$

I cocktailparty-problemet er kun de observerte totalsignalene $X$ kjent, og vi søker en invers matrise $W$, gjerne kalt *demiksematrisen*, til miksematrisen $A$ slik at $W X$ gir oss de opprinnelige kildesignalene.
Oppgaven virker tilsynelatende umulig, da vi hverken har kunnskap om $S$ eller $A$.
I praksis, derimot, viser det seg at vi kan oppnå overraskende gode estimater for $S$ gjennom en enkel antagelse om den statistiske distribusjonen av kildesignalene.

Vi vil her løse cocktail-party problemet med *uavhengig komponentanalyse* (engelsk: *independent component analysis (ICA)*).
Kjerneantagelsen i ICA er at de opprinnelige signalkomponentene $s_i(t)$ er **statistisk uavhengige**.
For å grundig forstå hvordan denne antagelsen legger grunnlaget for ICA-metoden, må vi se den i sammenheng med sentralgrenseteoremet fra statistikken.

La oss definere de stokastiske variablene $X_i$ og $S_i$ med fordelinger som representerer et utvalg av totalsignalene $x_i(t_j)$ og kildesignalene $s_i(t_j)$, henholdsvis.
Vi definerer så vektorene $\boldsymbol{X} = [X_1, \ldots, X_n]^T$ og $\boldsymbol{S} = [S_1, \ldots, S_n]^T$, slik at sammenhengen mellom distribusjonene blir $\boldsymbol{X} = A \boldsymbol{S}$.
Vi ønsker da å finne matrisen $W$ slik at $W \boldsymbol{X}$ gir oss tilbake distribusjonene av kildesignalene.
Hvilken egenskap må en rad $\boldsymbol{w} = [w_1, \ldots, w_n]$ i $W$ ha, gitt at kildesignalene $S_i$ er statistisk uavhengige?

La oss nå betrakte den stokastiske variabelen $Y = \sum_i w_i \cdot X_i = \boldsymbol{w} \boldsymbol{X}$.
Siden $\boldsymbol{w}$ er en rad i $W$, vil $Y$ representere et av kildesignalene $S_i$.
Ved å definere $\boldsymbol{z} = [z_1, \ldots, z_n] = \boldsymbol{w} A$, kan vi skrive $Y = \boldsymbol{w} A \boldsymbol{S} = \boldsymbol{z} \boldsymbol{S} = \sum_i z_i \cdot S_i$.
Altså er $Y$ en lineær kombinasjon av de uavhengige stokastiske variablene $S_1, \ldots, S_n$.
Fra sentralgrenseteoremet vet vi at en lineær kombinasjon av to eller flere uavhengige stokastiske variabler sannsynligvis er mer gaussisk fordelt enn de uavhengige variablene.
Dermed er $Y$ minst like gaussisk fordelt som $S_i$-ene, og den blir minst gaussisk fordelt når kun ett av elementene i $\boldsymbol{z}$ er forskjellig fra $0$.
For at $Y$ skal representere fordelingen til et av kildesignalene $S_i$, må vi derfor velge $\boldsymbol{w}$ slik at $\boldsymbol{w} \boldsymbol{X}$ er minst mulig gaussisk fordelt!

I denne notatboken vil vi implementere *FastICA*-algoritmen, utarbeidet av Aapo Hyvärinen ved Helsinki University of Technology for å løse cocktailparty-problemet.

FastICA-algoritmen kan oppsummeres **svært grovt** i følgende steg:
1. Preprossesér dataene $X$ ved å sentrere dem.
2. Preprossesér dataene $X$ ved å white dem.
3. Gjett en verdi $W_0$ for demiksematrisen.
4. Finn en bedre approksimasjon $W_k$ for demiksematrisen for $k = 1, 2, \ldots$ som gjør radene i $W_k X$ mer gaussisk fordelt enn i $W_{k-1} X$, inntil $W_k = W_{k-1} = W$.
5. Beregn de antatte kildesignalene gjennom transformasjonen $W X$ av totalsignalene.

Vi vil ha en anvendt tilnærming der vi motiverer og forklarer hensikten bak de ulike stegene i algoritmen mens vi implementerer den del for del, som beskrevet i et dokument av Brynjulf Owren.<sup>[1]</sup>
For detaljerte bevis for "korrektheten" til algoritmen, refererer vi til materiale produsert av Hyvärinen selv.<sup>[2]</sup>
Til slutt vil vi teste algoritmen på noen lydklipp.

I denne notatboken benytter vi noen ulike Python-biblioteker:

In [ ]:
import numpy as np # for effective numerical operations
import wav_file_loader # for loading data from .wav audio files
import IPython.display as ipd # for interactive audio playback
import time # for benchmarking

`wav_file_loader` vil utleveres sammen med notatboken, mens `numpy` og `IPython` anses som velkjente og lett tilgjengelige biblioteker.

## Preprosessering

I FastICA-algoritmen vil dataene $X$ først sendes gjennom et par preprosseseringssteg.
Hensikten med preprosseseringen er å gi dataene egenskaper som forenkler senere steg i algoritmen.

Først vil amplitudene i hvert lydopptak forskyves gjennom

$$\boldsymbol{x_i} \gets \boldsymbol{x_i} - E[\boldsymbol{x_i}]$$

der $\boldsymbol{x_i}$ er rad $i$ i $X$ og $E[\boldsymbol{x_i}]$ er forventningsverdien til elementene i raden, slik at dataene i radene er sentrert om sine middelverdier.

In [ ]:
def center_row_means(M):
    """Returns a new matrix in which the elements of each row in
    the matrix M are shifted so the mean of each row is zero.
    """
    n_rows = len(M)
    means = np.mean(M, axis=1).reshape((n_rows, 1))
    M -= means # subtract the mean of each row from all its elements
    return M

Deretter vil dataene sendes gjennom "whitening"-transformasjonen

$$ X \gets C^{-1/2} X$$

der $C = \frac{1}{N - 1} X X^T$ er en såkalt kovariansematrise til dataene $X$.
Med $C^{-1/2}$ mener vi en matrise som oppfyller $C^{-1/2} C^{-1/2} = C^{-1}$.
Vi finner denne såkalte kvadratroten til inversmatrisen ved hjelp av egenverdidekomponeringen $C^{-1/2} = E D^{-1/2} E^{-1}$, der $E$ er egenverdimatrisen til $C$ og $D^{-1/2}$ er en diagonal matrise med korresponderende inverse kvadratrøtter av egenverdiene til $C$ langs diagonalen.
Konstruksjonen av $C$ som en symmetrisk matrise garanterer eksistensen av reelle og positive egenverdier og gir i tillegg forenklingen $E^{-1} = E^T$.

Whitening-transformasjonen endrer dataene $X$ på en slik måte at ligningen $X = A S$ fortsatt skal løses under de samme forutsetningene som før, men mot en **ortogonal** matrise $A$, noe som gir ytterligere forenklinger senere i algoritmen.

In [ ]:
def symmetric_matrix_inverse_sqrt(M):
    """Returns a new matrix that is the square root of the inverse of the matrix M.
    The matrix M is assumed to be invertible and symmetric.
    """
    eigvals, eigvecs = np.linalg.eig(M)
    D = np.diag(1 / np.sqrt(eigvals))
    M_inv_sqrt = eigvecs @ D @ eigvecs.T
    return M_inv_sqrt

def whiten_matrix(X):
    """Returns a new whitened matrix from the matrix X."""
    n_cols = X.shape[1]
    C = (1 / (n_cols - 1)) * X @ X.T
    C_inv_sqrt  = symmetric_matrix_inverse_sqrt(C)
    X = C_inv_sqrt @ X 
    return X

## Iterasjon for demiksematrisen

Kjernen i FastICA-algoritmen er iterasjonen over matriser $W_k$ som maksimerer ikke-gaussiskheten til radene i $W_k X$.
Iterasjonen bestemmer en bedre approksimasjon $W_k$ til $W$ basert på en tidligere approksimasjon $W_{k-1}$, slik at radene i $W_k X$ er mer gaussisk fordelt enn i $W_{k-1} X$.
Vi understreker at dataene $X$ på dette stadiet i algoritmen er preprossesert i henhold til stegene over.

Først måles gaussiskheten til dataene i radene til $S_{k-1} = W_{k-1} X$ ved å la en velvalgt reell funksjon $g(u)$ og dens deriverte $g'(u)$ operere elementvis på $S_{k-1}$.
Resultatene lagres i matrisene $G$ og $G'$.
Vi åpner her for bruk av både kurtosefunksjonen $g_1(u) = 4 u^3$ og negentropifunksjonen $g_2(u) = u e^{-u^2/2}$.
Matrisene $G$ og $G'$ inneholder dermed informasjon om den "nåværende" gaussiskheten til dataene og "endringen" i gaussiskheten ved variasjon av elementer i matrisen $W_k$.
Basert på denne informasjonen, beregnes først en ny matrise

$$W_k^+ = \frac{1}{N} G X^T - \text{diag}(E[G']) W_{k-1}$$

der $\text{diag}(E[G'])$ er en diagonal $n \times n$ matrise med middelverdiene til radene i $G'$ langs diagonalen.
Så normaliseres radene i $W_k^+$ slik at den euklidske normen av hver rad er $1$.

Til slutt beregnes $W_k$ som den nærmeste ortogonale matrisen til $W_k^+$ gjennom transformasjonen
$$W_k = (W_k^+ (W_k^+)^T)^{-1/2} W_k^+$$

Kvadratroten av en invers matrise beregnes her på samme måte og under samme forutsetninger som før.

Vi har her delt opp denne prosedyren i et optimaliseringssteg og et ortogonaliseringssteg, også kjent som et dekorreleringssteg.

In [ ]:
def g_kurtosis(u):
    """Kurtosis"""
    return 4 * u**3

def dg_kurtosis(u):
    """Kurtosis derivative"""
    return 12 * u**2

def g_negentropy(u):
    """Negentropy"""
    return u * np.exp( -u**2 / 2)
    
def dg_negentropy(u):
    """Negentropy derivative"""
    return (1 - u**2) * np.exp( -u**2 / 2)    

def normalize_row_norms(M):
    """Returns a new matrix in which the rows of the matrix M 
    are normalized so their euclidean norm is 1.
    """
    n_rows = len(M)
    row_norms = np.linalg.norm(M, axis=1).reshape((n_rows, 1))
    M /= row_norms # divide each row by its norm
    return M

def optimize_demixing_matrix(W, X, g, dg):
    """Returns a new matrix W' so that the rows of W' * X are distributed more non-gaussianly
    than the rows of W * X.
    "Gaussianity" is measured using the scalar function g and its derivative, dg.
    The rows of the matrix X are assumed to have zero mean, and X is assumed to be whitened.
    """
    S = W @ X
    G = g(S)
    dG = dg(S)
    N = X.shape[1]
    W = (1 / N) * G @ X.T - np.diag(np.mean(dG, axis=1)) @ W
    W = normalize_row_norms(W)
    return W

def orthogonalize_matrix(W):
    """Returns a new matrix that is the nearest orthogonal matrix to the matrix W."""
    return symmetric_matrix_inverse_sqrt(W @ W.T) @ W

def update_demixing_matrix(W, X, g, dg):
    """Returns a new matrix that is the next iteration of the demixing matrix in the FastICA algorithm,
    given the previous demixing matrix W and the data matrix X.
    The rows of the matrix X are assumed to have zero mean, and X is assumed to be whitened.
    """
    W = optimize_demixing_matrix(W, X, g, dg)
    W = orthogonalize_matrix(W)
    return W

## FastICA-algoritmen

Nå som vi har funksjonalitet for å finne stadig bedre approksimasjoner til demiksematrisen $W$, er vi endelig klare til å sette sammen alle delene til FastICA-algoritmen som beskrevet over!

Algoritmen itererer over stadig bedre approksimasjoner $W_k$ for $W$ inntil to etterfølgende matriser $W_k$ og $W_{k-1}$ har konvergert innenfor en gitt toleranse eller iterasjonen er gjennomført et gitt maksimalt antall ganger.
Konvergens måles her med det maksimale avviket på diagonalen mellom identitetsmatrisen $I$ og $W_k W_{k-1}^T$.
Dette er mer effektivt i denne sammenhengen enn å, for eksempel, finne det maksimale elementvise avviket mellom $W_k$ og $W_{k-1}$, da algoritmen garanterer at $W_k$ og $W_{k-1}$ alltid er ortogonale, slik at $W_k W_k^T = W_{k-1} W_{k-1}^T = I$.

In [ ]:
def orthonormal_matrices_are_equal(A, B, tol):
    """Returns whether the matrices A and B are equal up to the tolerance tol. 
    A and B are assumed to be orthonormal matrices.
    The tolerance is compared with the maximum deviance between
    the identity matrix and A * B^T along their diagonals.
    """
    row_dot_products = np.sum(A * B, axis=1)
    max_dev = np.abs(np.max(1 - np.abs(row_dot_products)))
    return max_dev < tol

def fast_ICA(X, tol=1e-10, max_iters=100, g=g_kurtosis, dg=dg_kurtosis):
    """Returns new matrices S and W containing separated data components and the 
    demixing matrix W using the FastICA algorithm, given data from the matrix X,
    and the number of iterations, i, the algorithm uses to approximate W.
    The algorithm terminates after reaching max_iters iterations or when 
    two successive approximations of W are equal within the tolerance tol.
    """
    n = len(X)
    X = center_row_means(X)
    X = whiten_matrix(X)
    
    W = np.identity(n) # initial guess for W
    for i in range(0, max_iters):
        W0 = W
        W = update_demixing_matrix(W, X, g, dg) 
        if orthonormal_matrices_are_equal(W, W0, tol):
            break

    if i == max_iters - 1:
        print("warning: FastICA returned before convergence")
    S = W @ X
    return S, W, i

## Testing av FastICA-algoritmen

I denne seksjonen anvender vi FastICA-algoritmen på flere blandede lydopptak og sammenligner de separerte lydsignalene med de blandede opptakene og eventuelle kjente, opprinnelige kildesignaler.

For å forenkle denne testeprosessen, har vi utviklet et enkelt rammeverk som håndterer innlesing og avspilling av lyddata samt operasjoner på disse.
Rammeverket vil normalisere både blandede og separerte lydsignaler, slik at de oppfattes til å ha lik maksimal styrke.
Det har også funksjonalitet for å blande sammen lydsignaler på en tilfeldig måte.

Detaljene i rammeverket er på ingen måte nødvendig å undersøke for andre enn spesielt interesserte, da dets eneste hensikt er å forenkle og tydeliggjøre testingen av FastICA-algoritmen på forskjellige lydfiler og på ulike måter.
Hvordan testingen foregår vil gå klart frem av bruken av rammeverket.
Vi understreker allikevel for eventuelle skeptiske lesere at rammeverket alltid separerer blandede lydklipp med FastICA-algoritmen, enten de opprinnelige separerte lydklippene er kjente eller ikke.

In [ ]:
def normalize_audio(data):
    """Returns a new matrix in which the rows of the matrix M 
    are normalized so their maximum absolute values are 1.
    """
    n_tracks = len(data)
    row_maximums = np.amax(np.abs(data), axis=1).reshape((n_tracks, 1))
    data /= row_maximums # divide each row by its maximum absolute value
    return data

def random_mixing_matrix(signals, observations):
    """ Returns a new random mixing matrix.
    Each element is a small positive number, not too close to 0.
    """
    A = 0.25 + np.random.rand(observations, signals)
    return normalize_audio(A)

class Tester:
    """Object used to demonstrate and test the FastICA algorithm 
    on the .wav audio files located at given file paths.
    """
    def __init__(self, paths):
        data_observed, srate = wav_file_loader.read_wavefiles(paths)
        self.data_observed = normalize_audio(data_observed)
        self.data_separated = None
        self.srate = srate
        self.paths = paths
        self.set_tol(1e-9)
        self.use_kurtosis()

    def mix_observed(self):
        print("Mixing observed data")
        mixmat = random_mixing_matrix(len(self.paths), len(self.paths))
        self.data_observed = mixmat @ self.data_observed
        self.data_observed = normalize_audio(self.data_observed)
        
    def set_tol(self, tol):
        self.tol = tol
    
    def set_g(self, g):
        self.g = g[0]
        self.dg = g[1]
    
    def use_kurtosis(self):
        """Assumes g_kurtosis and dg_kurtosis defined."""
        self.set_g((g_kurtosis, dg_kurtosis))
        
    def use_negentropy(self):
        """Asssumes g_negentropy and dg_negentropy defined."""
        self.set_g((g_negentropy, dg_negentropy))
        
    def play_observed(self):
        print("Playing observed data:")
        self._play_data(self.data_observed)
        
    def play_separated(self, sample_percent_of_data=1):
        if sample_percent_of_data == 1:
            self._separate_data()
        else:
            # create sampler that selects a random sample of data
            sampler = lambda data_list: data_list[:, np.random.randint(data_list.shape[1],size = int(sample_percent_of_data*data_list.shape[1]))]
            self._separate_data(sampler)
            
        print("Playing separated data:")
        self._play_data(self.data_separated)
        
    def get_duration(self):
        return self.data_observed.shape[1] / self.srate
        
    def _play_data(self, data_list):
        for data in data_list:
            ipd.display(ipd.Audio(data, rate=self.srate))
    
    def _separate_data(self, sampler = lambda x: x):
        start = time.time()
        
        data_observed_sample = sampler(self.data_observed)
        percent_of_data = (data_observed_sample.shape[1] / self.data_observed.shape[1]) * 100
        
        print("Started separating data")
        print("  Number of tracks:    %d" % self.data_observed.shape[0])
        print("  Samples per track:   %d" % self.data_observed.shape[1])
        print("  Sample rate:         %d Hz" % self.srate)
        print("  Duration per track:  %.1f s" % self.get_duration())
        print("  Gaussianity measure: %s" % self.g.__doc__.split()[0].lower())
        print("  Tolerance:           %e" % self.tol)
        print("  Sample size:         %.2f%%" % percent_of_data)
        
        # do transformation of data ourselves
        _, W, iters = fast_ICA(data_observed_sample, tol=self.tol, g=self.g, dg=self.dg)  
        self.data_separated = whiten_matrix(center_row_means(self.data_observed))
        self.data_separated = W @ self.data_separated
        self.data_separated = normalize_audio(self.data_separated)
        
        end = time.time()
        
        print("Finished separating data")
        print("  Iterations:          %d" % iters)
        print("  Time used:           %.4f s" % (end - start))

Vi tester først FastICA-algoritmen på $n = 3$ utdelte blandede lydopptak av like mange lydkilder.
Testen gjøres både med kurtosevarianten og negentropivarianten av funksjonene som måler gaussiskhet.

Vi hører at lydsignalene separeres på en svært god måte, både ved bruk av kurtose og negentropi i målingen av gaussiskheten til data underveis i algoritmen.
Alle de separerte lydsporene er helt rene og meningsfulle lydspor og ingen av dem bærer preg av å være blandet med noen av de andre separerte lydsporene.
Lydsporene separeres like godt uavhengig av om de inneholder tale eller musikk.

In [ ]:
paths = ["audio/" + f + ".wav" for f in ("mix_1", "mix_2", "mix_3")]
test = Tester(paths)

test.play_observed()

test.use_kurtosis()
test.play_separated()

test.use_negentropy()
test.play_separated()

Vi illustrerer nå hvordan langvarige blandede lydklipp kan separeres svært effektivt ved å kun bruke et tilfeldig **utvalg** av totalsignalene i beregningen av demiksematrisen $W$, for å så benytte denne matrisen til å separere **hele** det blandede lydklippet.
Den naive metoden med å benytte hele lydklippet også i beregningen av $W$ fungerer like godt, men ikke merkbart bedre, og er betydelig tregere og krever mer minne.
Leseren kan selv sammenligne tidsbruken med den naive metoden på de samme lydklippene ved å fjerne kommentaren av linjen under som gjennomfører den naive metoden.

Intuitivt er det tenkelig at miksematrisen $A$, og dermed demiksematrisen $W$, avhenger av romlige og andre tidsuavhengige faktorer.
Dermed er det interessant, men ikke overraskende, at kun et utvalg av totalsignalene er nødvendig i bestemmelsen av $W$, da prosedyren bygger på statistiske prinsipper om distribusjoner av data.

In [ ]:
paths = ["audio/" + f + ".wav" for f in ("nwa1", "nwa2", "nwa3", "jimcarreymb")]
test_mix = Tester(paths)

test_mix.play_observed()
test_mix.mix_observed()
test_mix.play_observed()
#test_mix.play_separated() # commented by default
test_mix.play_separated(sample_percent_of_data=0.02)

Til slutt forsøker vi å benytte FastICA-algoritmen på flere, kortere lydklipp.
Vi benytter her 8 ulike lydklipp.
Her får vi best resultater om vi benytter negentropi og bruker hele lydklippene for å beregne demiksematrisen.

Vi oppfordrer leseren til å forsøke å benytte kurtose og å separere dataene kun med halvparten av datapunktene ved å fjerne kommentarene på en av de kommenterte linjene under eller begge.
I disse tilfellene overlapper flere av de separerte lydsporene på vår maskin.

In [ ]:
paths = ["audio/" + f + "_short.wav" for f in ("nwa1", "nwa2", "nwa3", "nwa4", "nwa5", "nwa6", "cubanpete", "jimcarreymb")]
test_many = Tester(paths)
test_many.use_negentropy()
#test_many.use_kurtosis() # commented by default

test_many.play_observed()
test_many.mix_observed()
test_many.play_observed()
test_many.play_separated()
#test_many.play_separated(sample_percent_of_data=0.5) # commented by default

## Konklusjon

Vi har i denne notatboken implementert FastICA-algoritmen for å løse cocktailparty-varianten av blind kildeseparasjonsproblemet med uavhengig komponentanalyse.
Separasjonsalgoritmen ble deretter testet på noen lydfiler av ulike antall, mengder og lengder ved bruk av både kurtose og negentropi.

I nesten alle tilfeller ble lydfilene separert bemerkelsesverdig godt.
Kun i ett tilfelle, der mange komponenter ble forsøkt separert med et tilfeldig utvalg datapunkter eller ved bruk av kurtose, feilet separasjonen ved at flere av sporene overlappet.
Men til og med her gikk separasjonen feilfritt ved bruk av negentropi på alle datapunktene på maskinen vår.
Kurtose og negentropi ser dermed ut til å fungere like godt og raskt til måling av gaussiskhet i de fleste tilfeller, men det virker som negentropi har muligheten til å oppnå større presisjon og kortere tidsbruk i enkelte mer krevende situasjoner.

Vi har merket at FastICA-algoritmen kan benyttes svært effektivt til separasjon av få lange lydklipp når kun et tilfeldig utvalg av datapunktene benyttes i beregningen av demiksematrisen.
Med et utvalg på kun 2% av datapunktene, separerte vi på denne måten fire 90-sekunders lydklipp feilfritt på under ett sekund på vår maskin!
Legg merke til at den desidert mest tidkrevende operasjonen i denne testen var innlastingen av lydavspillingsverktøyene.
Separasjon av de samme lydklippene med bruk av alle datapunktene produserte identiske resultater med betydelig større tidsbruk.
Denne metoden fungerte derimot ikke like godt når vi forsøkte å benytte den på mange korte lydklipp.
Her måtte vi benytte alle datapunktene for å få helt nøyaktige resultater, noe som tok betydelig lenger tid, selv om den totale lengden av lydklippene her var mye kortere.

Vi har også sett at Python i samspill med biblioteket `numpy` er svært velegnet for å implementere en enkel og lærerik versjon av algoritmen.
De effektive numeriske operasjonene i `numpy` og den enkle, tilgivende og intuitive syntaksen lot oss holde alt fokus på simpelthen å forflytte en matematisk beskrivelse av algoritmen over på datamaskinen.

For å gjøre en bedre vurdering av FastICA-algoritmen, kunne vi testet den på et enda mer variert utvalg lydklipp av flere ulike antall, lengder og typer.
Dette kunne avkreftet noen av konklusjonene vi har kommet frem til her, som kan være spesielle for de kombinasjonene av lydklipp vi har testet.
Vi kunne også latt algoritmen gjette en tilfeldig initiell verdi for demiksematrisen, fremfor å alltid benytte den samme identitetsmatrisen, for å se om dette påvirker konvergensen.
Vi kunne også forsket litt på å variere konvergenstoleransen i FastICA-algoritmen, men vi så ingen grunn til å gjøre dette her, da vi benyttet en svært lav toleranse i utgangspunktet og algoritmen alltid terminerte raskt nok når den ga korrekte separasjoner.

Til tross for noen små imperfeksjoner når FastICA-algoritmen kjøres med noen spesielle innstillinger på visse kombinasjoner av lydklipp, kan vi uansett ikke gjøre annet enn å si at FastICA-algoritmen ser ut til å fungere utrolig godt til separasjon av mange, lange og varierte lydklipp!

## Referanser
[1] B. Owren. Blind source separation. 2019.

[2] A. Hyvärinen, E. Oja. Independent component analysis: algorithms and applications. Neural Networks 12. 2000. Side 411-430.